# Pandas

## Carregamento e Armazenamento de Dados

### Arquivo CSV

In [2]:
import pandas as pd
poke = pd.read_csv('Pokemon.csv')
poke.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


### Arquivo Json

In [3]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
{"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""
print(type(obj))
print(obj)

<class 'str'>

{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
{"name": "Katie", "age": 33, "pet": "Cisco"}]
}



In [4]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
  {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}

In [5]:
asjson = json.dumps(result)
print(type(asjson))
asjson

<class 'str'>


'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}, {"name": "Katie", "age": 33, "pet": "Cisco"}]}'

In [6]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


### SQL

In [7]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);"""

In [8]:
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [9]:
data = [('Atlanta', 'Georgia', 1.25, 6),
('Tallahassee', 'Florida', 2.6, 3),
('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [10]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [11]:
print(cursor.description)
pd.DataFrame(rows, columns=list(zip(*cursor.description))[0])

(('a', None, None, None, None, None, None), ('b', None, None, None, None, None, None), ('c', None, None, None, None, None, None), ('d', None, None, None, None, None, None))


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [12]:
import pandas.io.sql as sql
sql.read_sql_query('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## Data Preparation

### Combinação de Dados

In [13]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd','b'],
                 'data2': range(4)})

pd.merge(df1,df2,on='key') #default inner

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,6,1
5,b,6,3
6,a,2,0
7,a,4,0
8,a,5,0


In [14]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [15]:
pd.merge(df3, df4, how='outer',left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [16]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [17]:
import numpy as np
arr = np.arange(12).reshape((3, 4))
print(arr)
np.concatenate([arr, arr], axis=1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [18]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [22]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), 
                index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), 
                index=['a', 'c'],
                columns=['three', 'four'])
print(df1)
print(df2)
pd.concat([df1, df2], axis=1, sort=False)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


### Transformação de Dados

#### Remoção de duplicatas

In [23]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
print(data)
print(data.duplicated())
data.drop_duplicates()

    k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4
0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool


,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [26]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
print(data)
data.drop_duplicates(['k1', 'k2'], keep='last')

    k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4


,k1,k2
1,one,1
2,one,2
4,two,3
6,two,4


In [27]:
data2 = pd.DataFrame({'food': ['bacon', 'costela suina', 'bacon', 'picanha',
                           'maminha', 'Bacon', 'picanha', 'presunto','sashimi'],
                  'porcao': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

print(data2)

            food  porcao
0          bacon     4.0
1  costela suina     3.0
2          bacon    12.0
3        picanha     6.0
4        maminha     7.5
5          Bacon     8.0
6        picanha     3.0
7       presunto     5.0
8        sashimi     6.0


In [29]:
meat_to_animal = {'bacon':'porco','costela suina':'porco','picanha':'boi',
                 'maminha':'boi','presunto':'porco','sashimi':"salmao"}

In [30]:
data2['animal'] = data2['food'].map(str.lower).map(meat_to_animal)
data2

,food,porcao,animal
0,bacon,4.0,porco
1,costela suina,3.0,porco
2,bacon,12.0,porco
3,picanha,6.0,boi
4,maminha,7.5,boi
5,Bacon,8.0,porco
6,picanha,3.0,boi
7,presunto,5.0,porco
8,sashimi,6.0,salmao


### Replace

In [31]:
data3 = pd.Series([1., -999., 2., -999., -1000., 3.])
print(data3)
data3.replace(-999,np.nan)
#data3.replace([-999,-1000],np.nan)

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64


0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

### Discretização e Binning

In [32]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
group_names = ['Jovem','Meia Idade','Adulto','Idoso']
cats = pd.cut(ages, bins, labels=group_names)
print(cats)
print(pd.value_counts(cats))

[Jovem, Jovem, Jovem, Meia Idade, Jovem, ..., Meia Idade, Idoso, Adulto, Adulto, Meia Idade]
Length: 12
Categories (4, object): [Jovem < Meia Idade < Adulto < Idoso]
Jovem         5
Adulto        3
Meia Idade    3
Idoso         1
dtype: int64


In [ ]:
poke.head()

## TODO Section

### Manipulação de Dados usando Pandas

Usando o dataset Pokemon.csv, faça:

    1) Verifique em qual(is) coluna(s) existem valores faltantes
    2) Preencha os valores faltantes da coluna Type 2 com os valores correspondentes da coluna Type 1
    3) Crie um DataFrame a partir dos dados originais contendo apenas pokemons lendários. Imprima os 5 primeiros
    4) Use apply/applymap para passar todos os valores das colunas Name, Type 1 e Type 2 para minúscula
    5) Agrupe os pokemons por Type 1 e retorne uma Série ordenada pela quantidade em ordem decrescente

In [34]:
# Preparação inicial

import pandas as pd
import numpy as np

poke = pd.read_csv('Pokemon.csv')

In [35]:
# resposta 1

print(poke)
#answer1 = pd

       #                   Name   Type 1  Type 2  Total  HP  Attack  Defense  \
0      1              Bulbasaur    Grass  Poison    318  45      49       49   
1      2                Ivysaur    Grass  Poison    405  60      62       63   
2      3               Venusaur    Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur    Grass  Poison    625  80     100      123   
4      4             Charmander     Fire     NaN    309  39      52       43   
..   ...                    ...      ...     ...    ...  ..     ...      ...   
795  719                Diancie     Rock   Fairy    600  50     100      150   
796  719    DiancieMega Diancie     Rock   Fairy    700  50     160      110   
797  720    HoopaHoopa Confined  Psychic   Ghost    600  80     110       60   
798  720     HoopaHoopa Unbound  Psychic    Dark    680  80     160       60   
799  721              Volcanion     Fire   Water    600  80     110      120   

     Sp. Atk  Sp. Def  Speed  Generatio

In [ ]:
# resposta 2
answer2 = poke
answer2.replace(np.nan, )

In [ ]:
# resposta 3


In [ ]:
# resposta 4


In [ ]:
# resposta 5
